### Ragas is a framework that helps you evaluate your Retrieval Augmented Generation (RAG) pipelines


In [2]:
#!pip install openai==0.28.1
#!pip install openai --upgrade
#!pip install ragas
#!pip install unstructured
#!pip install langchain[all]
#!pip install --upgrade langchain

#!pip install playwright
#!pip install -U selenium unstructured
#!pip install --upgrade langchain langchain-community langchainhub langchain-openai langchain-chroma bs4

In [3]:
#!pip install pydantic==2.5

In [1]:
import os, json
#import openai
#from langchain.chat_models import ChatOpenAI, ChatGooglePalm
from utils import OPENAI_API_KEY

os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY 
#os.environ["LANGCHAIN_TRACING_V2"] = "true"

#openai.api_key = os.environ['OPENAI_API_KEY']

In [2]:
import bs4
from langchain import hub
from langchain_community.document_loaders import SeleniumURLLoader, TextLoader
from langchain_chroma import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_openai import ChatOpenAI, OpenAIEmbeddings


In [27]:


llm = ChatOpenAI(model="gpt-4-0125-preview", temperature=0.)

In [28]:

def get_questioins(path):
    loader = TextLoader(path)
    docs = loader.load()
    texts = docs[0].page_content.split('\n')
    questions = []
    for q in  texts:
        if "?" in q:
            questions.append(q)
    return questions

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

def load_url_documets(list_urls):
    
    # Load, chunk and index the contents of the blog.
    loader_url =SeleniumURLLoader( list_urls)
    docs = loader_url.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    splits = text_splitter.split_documents(docs)
    
    # Retrieve and generate using the relevant snippets of the blog.
    vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())   
    retriever = vectorstore.as_retriever()
    return retriever, docs

In [29]:
qa_dict = {}

url_list = ["https://www.nature.com/articles/s41524-023-01062-z",
            "https://www.nature.com/articles/s41699-023-00369-1",
            "https://github.com/HSE-LAMBDA/ai4material_design/tree/main/docs/CONSTRUCTOR-MOCK.md"
            "https://github.com/HSE-LAMBDA/ai4material_design/blob/main/docs/CONSTRUCTOR.md",
            "https://github.com/HSE-LAMBDA/ai4material_design/blob/main/docs/DATA.md",
            "https://github.com/HSE-LAMBDA/ai4material_design/blob/main/docs/ENVIRONMENT.md",
            "https://github.com/HSE-LAMBDA/ai4material_design/blob/main/docs/GENERATING-CONSTRUCTOR.md",
            "https://github.com/HSE-LAMBDA/ai4material_design/blob/main/docs/GENERATING-MOCK.md",
            "https://github.com/HSE-LAMBDA/ai4material_design/blob/main/docs/PILOT.md",
            "https://github.com/HSE-LAMBDA/ai4material_design/blob/main/docs/SPARSE-PAPER.md"
          #  "https://www.nature.com/articles/s41377-024-01407-3",
          #  "https://www.nature.com/articles/s41565-023-01407-1",
          #  "https://www.nature.com/articles/s41699-023-00369-1",
           ]
                               
retriever, documents = load_url_documets(url_list)

In [30]:
len(documents)

9

In [31]:
### Contextualize question ###
contextualize_q_system_prompt = """Given a chat history and the latest user question \
which might reference context in the chat history, formulate a standalone question \
which can be understood without the chat history. Do NOT answer the question, \
just reformulate it if needed and otherwise return it as is."""
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

### Answer question ###
qa_system_prompt = """You are an assistant for question-answering tasks. \
Use the following pieces of retrieved context to answer the question. \
If you don't know the answer, just say that you don't know. \
Use three sentences maximum and keep the answer concise.\

{context}"""
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)


In [32]:
### Statefully manage chat history ###
store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [33]:
from langchain.document_loaders import TextLoader
ques = get_questioins("data/data_rag/Sparse representation - questions.txt")

In [34]:
qa_dict = {}

for q in ques:
    answer = conversational_rag_chain.invoke(
              {"input": q},
               config={"configurable": {"session_id": "abc123"}
              },  # constructs a key "abc123" in `store`.
             )["answer"]
    qa_dict[q] = answer

In [35]:
l_answer, l_question = [], []

for k,v in qa_dict.items():
    print(k)
    print()
    print(v)
    print("**********************************")
    print()
    l_answer.append(v)
    l_question.append(k)
        

* Which materials are in the dataset?

The materials in the dataset include MoS2, WSe2, hBN, GaSe, InSe, and black phosphorous.
**********************************

* How many structures are there in the dataset?

The dataset contains structures with 5933 defect configurations for MoS2 and another 5933 configurations for WSe2, making a total of 11,866 defect configurations for these two materials. Additional structures for hBN, GaSe, InSe, and black phosphorus are included, but the exact numbers for these materials are not specified.
**********************************

* How to obtain the dataset?

The dataset, including DFT-computed properties, relaxed atomic structures, density of states (DOS), and band structures at the DFT PBE levels, is available for further studies and machine learning training, but the specific method or platform for obtaining the dataset is not mentioned in the provided context.
**********************************

* What is the dataset license?

The provided con

In [36]:
with open('data/data_rag/qa_dict_9docs&chatHistory_gpt-4-0125-preview.json', 'w') as fp:
    json.dump(qa_dict, fp)

In [37]:
for m in conversational_rag_chain.get_session_history("abc123").messages:
    
    if m.type == 'human':
        print('Human: ' + m.content)
    elif  m.type == 'ai':
        print('AI: ' + m.content)
        print()

Human: * Which materials are in the dataset?
AI: The materials in the dataset include MoS2, WSe2, hBN, GaSe, InSe, and black phosphorous.

Human: * How many structures are there in the dataset?
AI: The dataset contains structures with 5933 defect configurations for MoS2 and another 5933 configurations for WSe2, making a total of 11,866 defect configurations for these two materials. Additional structures for hBN, GaSe, InSe, and black phosphorus are included, but the exact numbers for these materials are not specified.

Human: * How to obtain the dataset?
AI: The dataset, including DFT-computed properties, relaxed atomic structures, density of states (DOS), and band structures at the DFT PBE levels, is available for further studies and machine learning training, but the specific method or platform for obtaining the dataset is not mentioned in the provided context.

Human: * What is the dataset license?
AI: The provided context does not specify the license under which the dataset is dist

In [38]:
import pandas as pd


l_answer, l_question = [], []

for m in conversational_rag_chain.get_session_history("abc123").messages:
    
    if m.type == 'human':
        l_question.append(m.content)
    elif  m.type == 'ai':
        l_answer.append(m.content)

df = pd.DataFrame()
df['question'] = l_question
#df['Content'] = doc_url[0].metadata['description']
df['answer'] = l_answer
df['number of documets'] = len(documents)
#for i in range(len(documents)):
    #df[f'Content_{i}'] = documents[i].metadata['description']
    #df[f'Titles_{i}'] = documents[i].metadata['title']
    #df[f'Urls_{i}'] = documents[i].metadata['source']

df.to_csv("data/data_rag/9docs&chatHistory_gpt-4-0125-preview.csv", index=False)